In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
from tqdm import tqdm
from datetime import datetime
import json
from sklearn.preprocessing import LabelEncoder
import calendar
import spotipy
warnings.filterwarnings("ignore")

# Web scrapping data from Spotify Chart

## Get date needed for each week

format of links to download:
https://spotifycharts.com/regional/global/weekly/2016-12-23--2016-12-30/download
https://spotifycharts.com/regional/global/weekly/2016-12-30--2017-01-06/download

We see that all the dates that appear in the link are Fridays, each being one week apart

In [ ]:
#An example of Calendar package
x = []

for month in range(1, 13):
    mycal = calendar.monthcalendar(2025, month)
    week1 = mycal[1]
    week2 = mycal[2]
      
    if week1[calendar.MONDAY] != 0:
        auditday = week1[calendar.MONDAY]
    else: 
        auditday = week2[calendar.MONDAY] 
    print("%10s %2d" % (calendar.month_name[month], auditday))
    x.append("%10s %2d" % (calendar.month_name[month], auditday))

In [ ]:
# reference
c = calendar.TextCalendar(calendar.MONDAY)
str1 = c.formatmonth(2017,1)
print(str1)

In [ ]:
(calendar.monthcalendar(2017,1)[0][calendar.FRIDAY],calendar.monthcalendar(2017,1)[4][calendar.FRIDAY])
#calendar.monthcalendar(2017,1)[c(0,4)][calendar.FRIDAY]

In [ ]:
#All the fridays in 2017
Fridays17 = []

for month in range(1, 13):
    mycal = calendar.monthcalendar(2017, month)
    
    for week in range(0,len(mycal)):
        if mycal[week][calendar.FRIDAY] != 0:
            #print(2017, month, mycal[week][calendar.FRIDAY])
            Fridays17.append((2017, month, mycal[week][calendar.FRIDAY]))
            
            
Fridays17[0]

In [ ]:
#All the fridays in 2018
Fridays18 = []

for month in range(1, 13):
    mycal = calendar.monthcalendar(2018, month)
    
    for week in range(0,len(mycal)):
        if mycal[week][calendar.FRIDAY] != 0:
            #print(2018, month, mycal[week][calendar.FRIDAY])
            Fridays18.append((2018, month, mycal[week][calendar.FRIDAY]))


Fridays18[0][2]

In [ ]:
#fridays in 2019 (Jan to Aug)
Fridays19 = []

for month in range(1, 9):
    mycal = calendar.monthcalendar(2019, month)
    
    for week in range(0,len(mycal)):
        if mycal[week][calendar.FRIDAY] != 0:
            #print(2019, month, mycal[week][calendar.FRIDAY])
            Fridays19.append((2019, month, mycal[week][calendar.FRIDAY]))

Fridays19[2]

In [ ]:
#Fridays in 2016
Fridays16to19 = [(2016, 12, 23),(2016, 12, 30)]

#Compile all the fridays 2016-2019
Fridays16to19 = Fridays16to19 + Fridays17 + Fridays18 + Fridays19
Fridays16to19[0]

In [ ]:
#formatting friday dates: 2016-12-23--2016-12-30

Fridays_format = []

for i in range(0, len(Fridays16to19)):
    year = Fridays16to19[i][0]
    month = Fridays16to19[i][1]
    day = Fridays16to19[i][2]  
    
    if month < 10:
        month = str(0)+str(month)
    else:
        month = str(month)
        
    if day < 10:
        day = str(0)+str(day)
    else:
        day = str(day)
    
    #print(str(year)+"-"+month+"-"+day)
    Fridays_format.append(str(year)+"-"+month+"-"+day)
    
Fridays_format[0]

In [ ]:
#put all the dates into the format of download link: 
#https://spotifycharts.com/regional/global/weekly/2016-12-23--2016-12-30/download

####################IndexError: list index out of range
# for j in range(0, len(Fridays_format)):
#     str_p1 = "https://spotifycharts.com/regional/global/weekly/" + Fridays_format[j] + "--"
#     if j != len(Fridays_format):
#         print(str_p1 + Fridays_format[j+1] + "/download")
        
# for j in range(0, len(Fridays_format)):
#     print("https://spotifycharts.com/regional/global/weekly/" + Fridays_format[j] 
#           + "--" + Fridays_format[j+1] + "/download")


final_df = pd.DataFrame(columns=['Position', 'Track', 'Artist', 'Streams', 'URL', 'Year', 'Month', 'Day’])
for first_date, second_date in zip(Fridays_format, Fridays_format[1:]):
    # Get url:
    url = "https://spotifycharts.com/regional/global/weekly/\" + first_date + \"--\" + second_date +\"download\"
     data = io.StringIO(requests.get(url).text)
    file = requests.get(url, allow_redirects=True)
    data = io.StringIO(requests.get(url).text)
    df = pd.read_csv(data, skiprows=1, error_bad_lines=False)
    # some cleaning:
    print(df.columns)
    df = df.iloc[:, [0,1,2,3,4]]
    df.columns = ['Position', 'Track', 'Artist', 'Streams', 'URL']
    print(df.columns)
    print(df['Track'])
    df['Track'] = df['Track'].apply(lambda x: x.strip('\\\"'))
    df['Artist'] = df['Artist'].apply(lambda x: x.strip('\\\"'))

    df['Year'] = second_date.split('-')[0]
    df['Month'] = second_date.split('-')[1]
    df['Day'] = second_date.split('-')[2]
                                 
    final_df = pd.concat([final_df, df], sort=False)


In [ ]:
final_df = final_df.drop(columns=['Track', '<html>'])
# final_df.to_csv('spotify_charts.csv')

## Billboard Dataset

In [ ]:
import csv
import pandas as pd

billboard_df = pd.read_csv("~/desktop/spotify_top_tracks_analysis/data/billboard_lyrics_1964-2015.csv",
                   encoding='latin-1')
billboard_df.head()

#billboard_df.describe()

In [ ]:
billboard_df.iloc[[4],[4]]

In [ ]:
billboard_df

## spotipy

In [ ]:
#setting up spotipy
import sys
import spotipy
import spotipy.util as util

scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print ("Usage: %s username" % (sys.argv[0],))
    sys.exit()
    
token = util.prompt_for_user_token(username,scope,client_id='ff9ff248e80e428bbb9796e1a7d62aeb',
                                   client_secret='7f70a7c884494c6c96d2e6f03c4d2388', redirect_uri='http://localhost/')

In [ ]:
sp = spotipy.Spotify(auth=token)

In [ ]:
name = 'Shawn Mendes'

results = sp.search(q='artist:' + name, type='artist')
print(results)

### Feature Engineering

In [ ]:
# try multiple assignment:
import pandas as pd
df_list = [[1,2,3], [4,5,6],[8,9,9]]
df = pd.DataFrame(df_list)
df

In [ ]:
df.columns = ['one', 'two', 'three']

In [ ]:
def get_variables(num):
    return num+1, num+2, num+3, num+4

In [ ]:
import numpy as np
column_names = ['four', 'five', 'six', 'seven']
pd.concat([df,pd.DataFrame(columns=column_names)], sort=False)

In [ ]:
df[column_names] = df['one'].apply(lambda x: pd.Series(get_variables(x)))

In [ ]:
df['four'] = None
df['five'] = None
df['six'] = None

In [ ]:
df